In [6]:
# -*- coding: utf-8 -*-
#Initial Imports
import gzip#; print('gzip version: ', gzip.__version__)
import os#; print('os version: ', os.__version__)
import re#; print('re version: ', re.__version__)
import shutil#; print('shutil version: ', shutil.__version__)
import requests#; print('requests version: ', requests.__version__)
import wget#; print('wget version: ', wget.__version__)
import subprocess#; print('subprocess version: ', subprocess.__version__)
import time#; print('time version: ', time.__version__)
from datetime import datetime
from subprocess import PIPE, Popen
import os.path
from os import path
from Bio.ExPASy import Enzyme
import numpy as np 

### Creating Unitprot.fasta for Use in Diamond

In [3]:
def tsv_to_fasta():
    reference_library = 'uniprot.tsv' 
    #this creates a file named 'uniprot.tsv' on the working directory. Should show within the EcoDr/EcoDr folder. 
    ua = UserAgent()
    header = {'User-Agent': str(ua.chrome)}
    uniprot_url = 'https://rest.uniprot.org/uniprotkb/stream?fields=accession%2Cec%2Csequence&format=tsv&query=%28%28ec%3A*%29%29+AND+%28reviewed%3Atrue%29'
    time.sleep(4)
    uniprot = requests.get(uniprot_url, headers=header)
    if uniprot.status_code == 200:
        with open(reference_library, 'w+') as reference_library:
            reference_library.write(uniprot.text)
    print('Uniprot Reference File Has Been Created')
####this step takes it the initial tsv and converts it to FASTA
    input = os.path.abspath('uniprot.tsv')
    output = 'uniprot.fasta'
    with open(input, 'r') as input_file, open(output, 'w') as output_file:
        header=next(input_file)
        for line in input_file:
            carrot = f'>{line}'
            new_row = re.sub(r'(.*?\t.*?)\t', r'\1\n', carrot, 1)
            new_row_with_tab_replaced_by_question_mark = new_row.replace("\t", "?")
            # Be careful with the symbols used for replacement, as they have to align with the genomic summary string parsing
            output_row_with_space_replaced_with_ampersand = new_row_with_tab_replaced_by_question_mark.replace("; ", ";_")
            #new_row2 = re.sub(r'(.*?\t.*?)\t', r'$', new_row, 0)
            #this is regex, for the record. 
            output_file.write(output_row_with_space_replaced_with_ampersand)
    print('FASTA has been created from TSV and is named', output)

In [4]:
tsv_to_fasta()

Uniprot Reference File Has Been Created
FASTA has been created from TSV and is named uniprot.fasta


### EC List Creation

In [5]:
def EC_extract():
    ec_library = 'EC_library.csv' 
    ua = UserAgent()
    header = {'User-Agent': str(ua.chrome)}
    ec_url = 'https://ftp.expasy.org/databases/enzyme/enzyme.dat'
    time.sleep(4)
    ec = requests.get(ec_url, headers=header)
    if ec.status_code == 200:
        with open(ec_library, 'w+', newline='\n') as ec_file:
            ec_file.write(ec.text)
###This step creates the list 
    handle = open(ec_library)
    records = Enzyme.parse(handle)
    ecnumbers = [record["ID"] for record in records]
    #print(type(ecnumbers)) #This is a list at this point in the code
    path = os.path.abspath(ec_library)
    with open(path, 'w+', newline='\n') as csv_file:
        #csv_file = csv.writer(csv_file)
        for item in ecnumbers:
            csv_file.write(item +'\n')
    print('EC list Has Been Created')

In [6]:
EC_extract()

EC list Has Been Created


### Diamond

In [21]:
def diamond_impl(dest, name):
    print(os.getcwd())
    matches = ''
    output_folder = dest 
    print("DIAMOND search library is: ", output_folder)
    if os.path.isfile('reference.dmnd'):
        print("Library Detected")
    # If not present, then creates a DIAMOND library by referencing the exact location where the Uniprot library is saved
    # If there currently is no reference library (.dmnd), then command makedb creates a DIAMOND library
    else:
        print("Creation of DIAMOND-formatted library...")
        makedb = ['diamond', 'makedb', '--in', '/home/anna/Documents/EnCen/uniprot.fasta', '-d',
                  'Uniprot_Reference_Library.dmnd']  # Reference library full pathway
        #This is a list for the DIAMOND specific makedb function. 
        subprocess.run(makedb)
        #This simply runs the function makedb
        print("Library complete")
#^this chunk is good to go. Just makes the reference database from the uniprot fasta. 
##_______________________________________________________________# This portion does the matching. The above portion creates the reference library from the unitprot fasta
    for item in os.listdir(dest):
        # Checks for file extension
        if item.endswith('.faa') and not os.path.isfile(
                os.path.basename(os.path.abspath(item)).rsplit('.', 1)[0] + "_matches.tsv"):
            # Finds path of file
            file_path = os.path.abspath(item)
            # Finds the GCF/ASM name of the file by looking at the first part of the name before the .faa notation
            if name == "":
                print(os.path.basename(file_path).rsplit('.',1))
                file_name = (os.path.basename(file_path)).rsplit('.', 1)[0]
            else:
                file_name = name
            # New filename that ends with matches
            matches = file_name + "_matches.tsv"
            print(matches)
            # If genome has not already undergone DIAMOND search and is currently located in the correct folder, then
            # the subprocess function will run the diamond search


            
            if not os.path.isfile(dest + "/" + matches) and os.path.abspath(matches) != output_folder:
                print("Processing ", file_name)
                # DIAMOND search using the full pathway of the protein files, max target sequence outputs only one best
                # match with highest e-value which represent the chance of obtaining a better random match in the same database (Buchfink et al, 2021)
                blastp = ['diamond', 'blastp', '-d', 'Uniprot_Reference_Library.dmnd', '-q', file_path, '-o', matches,
                          '--max-target-seqs', '1', '--outfmt', '6']
                time.sleep(4)
                subprocess.run(blastp)
        # (2) Creates a folder for DIAMOND outputs
        #if not os.path.exists(output_folder):
            #os.makedirs('DIAMOND_matches')

    # Moves all DIAMOND search outputs into the folder
        if item.endswith('_matches.tsv'):
            if os.path.exists(os.path.join(output_folder, item)):
                print(f"Overwriting: {item}")
                os.remove(os.path.join(output_folder, item))
            shutil.move(os.path.abspath(item), output_folder)
    print("diamond_impl--success")
    # Returns the location of the DIAMOND matches folder
    return output_folder

In [15]:
def genome_extractor(diamond_folder, name):
    os.chdir(diamond_folder)
    print(os.getcwd())
    # Opens the list of of EC numbers
    ec_open = np.loadtxt('/projects/jodo9280/EcoDr/EcoDr/EC_library.csv',
                         dtype='str')
    big_matrix = ["Name_of_Genome"]
    # Asks user to input name for EC matrix
    # input_name = input("Save EC binary summary matrix as (no spaces or capital letters): ")
    # Specifies document to be a csv type
    # file_name= input_name+".csv"
    # file_name = "synbio1_big_matrix.csv"
    if name == "":
        file_name = os.path.abspath(diamond_folder).rsplit('/', 9)[4] + '_binary_matrix.txt'
        print(os.path.abspath(diamond_folder).rsplit('/', 9))
        print(file_name)
    else:
        file_name = name
    new_dir = diamond_folder + '/' + file_name
    # Checks to see if the document already exists using full pathway name
    if os.path.exists(new_dir):
        pass
        #print("Summary Matrix exists")
        #return [new_dir, file_name]
    else:
        for ec_force in ec_open:
            # Creates a horizontal header of all of the EC names
            big_matrix.append(ec_force)
        # Goes through all of the DIAMOND outputs in the folder
        # Goes through all of the output files, each one is opened and read one line at a time. The lines are split to
        # extract the EC numbers found in each line. If the EC number found in the DIAMOND output matches an
        # EC entry in the list, the status is changed from a zero to a one. The binary status is catalogued horizontally
        # for each genome, and following genomes are vertically stacked
        for item in os.listdir(diamond_folder):
            if item.endswith("_matches.tsv"):
                print(item)
                # Finds the name of the DIAMOND output file
                genome = [item] #Turns the GCF's into a list, where the GCF names in the matrix come from. 
                genome_runner_ec = [item] #Turns the GCF's into a list, where the EC is appended
                # Iterates through all of the EC numbers (1:8197)
                
                GCF = open(item, 'r') # CBM Added
                
                for line in GCF: # CBM Added
                    no_tab = line.split('\t')
                    first_ec = no_tab[1].split("?")
                    separate_ec = first_ec[1].split(";_")
                    genome_runner_ec.append(separate_ec)
                    print("Appending...")

                for ec in ec_open:
                    # print("EC we actually are looking for "+ ec)
                    # Opens individual DIAMOND output files
                    #CBM GCF = open(item, 'r')
                    # Sets default for EC status is zero, meaning absent
                    #CBM ec_now = 0
                    # Takes the first line in the DIAMOND output file and splits it based on tab separation
                    # Takes the second column of the split line, which has EC numbers separated by a ?, ;_
                    # Strings splits have a new name assigned to them
                    #CBM for line in GCF:
                    #CBM    print(line)
                    #CBM    no_tab = line.split('\t')
                    #CBM    first_ec = no_tab[1].split("?")
                    #CBM    separate_ec = first_ec[1].split(";_")
                    #CBM    print("Seperate EC Likely Nightmare "+ separate_ec[0])
                        # Checks for a full match between the EC number listed in the DIAMOND output and the EC number
                        # found in the separate document
                    #CBM    if re.fullmatch(ec, first_ec[1]) is not None:  # looks for full match of first EC number
                    #CBM        ec_now = 1
                        # In the case that there are more than one EC separated by ;, the function iterates through the list
                        # and sees if there is a full match between the listed EC and the list
                    #CBM    for i in separate_ec:
                    #CBM        if re.fullmatch(ec, i) is not None:  # looks for full match of any other ECs listed
                    #CBM            ec_now = 1
                    ec_now = 0
                    if [ec] in genome_runner_ec:
                        ec_now = 1

                    # 1 or 0 will be appended to the summary matrix for each EC value in the list
                    genome.append(ec_now)
                    #print(genome)
                # Vertical stacking occurs for each genome in the DIAMOND output folder
                big_matrix = np.vstack([big_matrix, genome])
        #print(big_matrix)
        # Saves matrix as a text file for further analysis
        np.savetxt(file_name, big_matrix, fmt='%s')
        # Returns the location of the summary matrix and the name of the file
        if not os.path.exists('/projects/jodo9280/EcoDr/EcoDr/EcoDr_binary_matrix.txt'):
            shutil.move(os.path.abspath('EcoDr_binary_matrix.txt'),'/projects/jodo9280/EcoDr/EcoDr')
        else:
            print('File already Exists')
        print(new_dir)
        return [new_dir, file_name]

### Calling Script

In [26]:
print('I know things beyond the Maginot line are harried. \n But I worry. I\'ve asked you time and time again. \n  Abandon your cataloging. \n Come Back Inside, where my fleets can keep you safe')
print('Come Home')
metagenome_name = 'metagenome_analysis_output' #-> folder
desired_location = '/home/anna/Documents/JGI_soil_genomes' 
# Freshwater = nump/nump/nump/files
soil = '/home/anna/Documents/JGI_soil_genomes'
abspath = os.path.abspath(soil)



#______________________________________________________________#
os.chdir(desired_location) #-> we are in the folder we want
if os.path.exists(metagenome_name):
    shutil.rmtree(metagenome_name)
    os.mkdir(metagenome_name)
else:#makes a new directory called metagenome_name
    os.mkdir(metagenome_name)
desired_location = desired_location + "/" + metagenome_name #-> this will make a folder called 'metagenome_analysis_output' within the 'desired_location' 

# shutil.copy(abspath, desired_location) #moves  file to Test Cases folder
#matches = metagenome + "_matches"

I know things beyond the Maginot line are harried. 
 But I worry. I've asked you time and time again. 
  Abandon your cataloging. 
 Come Back Inside, where my fleets can keep you safe
Come Home


In [27]:
diamond = diamond_impl(soil, '') #-> Takes in the path and directory
# new_dir, saved_file_name = genome_extractor(desired_location, '') #-> At this point, we have the one hotted binary. 

/home/anna/Documents/JGI_soil_genomes
DIAMOND search library is:  /home/anna/Documents/JGI_soil_genomes
Creation of DIAMOND-formatted library...


diamond v0.9.30.131 (C) Max Planck Society for the Advancement of Science
Documentation, support and updates available at http://www.diamondsearch.org

#CPU threads: 20
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Database file: /home/anna/Documents/EnCen/uniprot.fasta
Opening the database file...  [0s]
Loading sequences...  [0.549s]
Masking sequences...  [0.993s]
Writing sequences...  [0.149s]
Hashing sequences...  [0.067s]
Loading sequences...  [0s]
Writing trailer...  [0.003s]
Closing the input file...  [0s]
Closing the database file...  [0s]
Database hash = 9ae9565aeffc045297d8444896762120
Processed 275960 sequences, 114276793 letters.
Total time = 1.763s


Library complete
['2162886007_5', 'faa']
2162886007_5_matches.tsv
Processing  2162886007_5


diamond v0.9.30.131 (C) Max Planck Society for the Advancement of Science
Documentation, support and updates available at http://www.diamondsearch.org

#CPU threads: 20
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Temporary directory: 
Opening the database...  [0s]
#Target sequences to report alignments for: 1
Reference = Uniprot_Reference_Library.dmnd
Sequences = 275960
Letters = 114276793
Block size = 2000000000
Opening the input file...  [0s]
Opening the output file...  [0s]
Loading query sequences...  [0.005s]
Masking queries...  [0.011s]
Building query seed set...  [0.01s]
Algorithm: Double-indexed
Building query histograms...  [0.004s]
Allocating buffers...  [0s]
Loading reference sequences...  [0.148s]
Masking reference...  [0.95s]
Initializing temporary storage...  [0s]
Building reference histograms...  [0.32s]
Allocating buffers...  [0s]
Processing query block 0, reference block 0, shape 0, index chunk 0.
Building reference seed array...  [0.221s]


['3300000532_1', 'faa']
3300000532_1_matches.tsv
Processing  3300000532_1


diamond v0.9.30.131 (C) Max Planck Society for the Advancement of Science
Documentation, support and updates available at http://www.diamondsearch.org

#CPU threads: 20
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Temporary directory: 
Opening the database...  [0s]
#Target sequences to report alignments for: 1
Reference = Uniprot_Reference_Library.dmnd
Sequences = 275960
Letters = 114276793
Block size = 2000000000
Opening the input file...  [0s]
Opening the output file...  [0s]
Loading query sequences...  [0.004s]
Masking queries...  [0.009s]
Building query seed set...  [0.007s]
Algorithm: Double-indexed
Building query histograms...  [0.002s]
Allocating buffers...  [0s]
Loading reference sequences...  [0.186s]
Masking reference...  [0.73s]
Initializing temporary storage...  [0s]
Building reference histograms...  [0.308s]
Allocating buffers...  [0s]
Processing query block 0, reference block 0, shape 0, index chunk 0.
Building reference seed array...  [0.229s

diamond_impl--success


 [0.457s]
Deallocating reference...  [0.012s]
Loading reference sequences...  [0s]
Deallocating buffers...  [0s]
Deallocating queries...  [0s]
Loading query sequences...  [0s]
Closing the input file...  [0s]
Closing the output file...  [0s]
Closing the database file...  [0s]
Deallocating taxonomy...  [0s]
Total time = 4.176s
Reported 1119 pairwise alignments, 1124 HSPs.
1119 queries aligned.
